In [1]:
import pandas as pd
import os
import torch
from model.database_util import *

In [2]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)

## Define Model

In [3]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 64
    # bs = 10
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [4]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [5]:
from model.dataset import PlanTreeDataset

## TEST - Loading 10 tensors

In [ ]:
import os
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from model.dataset import PlanTreeDataset  # Assuming PlanTreeDataset is defined elsewhere
import json

# Path to the tensors folder
tensors_dir = "./job_queries/tensors"

# Validate if the tensors directory exists
if not os.path.exists(tensors_dir):
    raise FileNotFoundError(f"Tensors directory '{tensors_dir}' not found.")

# Get all tensor file paths from the directory
tensor_files = sorted(os.listdir(tensors_dir))

# Validate if tensor files exist in the directory
if not tensor_files:
    raise FileNotFoundError(f"No tensor files found in '{tensors_dir}'.")

# Initialize lists to store tensor components
x_list = []
rel_pos_list = []
attn_bias_list = []
heights_list = []
cost_labels_list = []
raw_costs_list = []

# Load tensors dynamically
for tensor_file in tensor_files:
    tensor_path = os.path.join(tensors_dir, tensor_file)
    try:
        loaded_tensors = torch.load(tensor_path)
        # Append components to respective lists
        x_list.append(loaded_tensors["x"])
        rel_pos_list.append(loaded_tensors["rel_pos"])
        attn_bias_list.append(loaded_tensors["attn_bias"])
        heights_list.append(loaded_tensors["heights"])
        cost_labels_list.append(loaded_tensors["cost_labels"])
        raw_costs_list.append(loaded_tensors["raw_costs"])
    except Exception as e:
        print(f"Error loading tensor file '{tensor_file}': {e}")

# Generate indices for splitting
num_examples = len(x_list)
if num_examples == 0:
    raise ValueError("No valid tensors loaded for dataset creation.")
    
all_indices = np.arange(num_examples)

# Perform train-validation split with fixed seed for reproducibility
train_indices, val_indices = train_test_split(all_indices, test_size=0.2, random_state=0)

# Create training and validation datasets
train_dataset = PlanTreeDataset(
    len(train_indices),
    [x_list[i] for i in train_indices],
    [attn_bias_list[i] for i in train_indices],
    [rel_pos_list[i] for i in train_indices],
    [heights_list[i] for i in train_indices],
    [cost_labels_list[i] for i in train_indices],
    [raw_costs_list[i] for i in train_indices]
)

val_dataset = PlanTreeDataset(
    len(val_indices),
    [x_list[i] for i in val_indices],
    [attn_bias_list[i] for i in val_indices],
    [rel_pos_list[i] for i in val_indices],
    [heights_list[i] for i in val_indices],
    [cost_labels_list[i] for i in val_indices],
    [raw_costs_list[i] for i in val_indices]
)

# Save validation indices and file names
val_file_names = [tensor_files[i] for i in val_indices]
val_data = {
    "val_indices": val_indices.tolist(),
    "file_names": val_file_names,
}

val_data_file = "./val_data.json"
with open(val_data_file, "w") as f:
    json.dump(val_data, f)

# Save training indices and file names
train_file_names = [tensor_files[i] for i in train_indices]
train_data = {
    "train_indices": train_indices.tolist(),
    "file_names": train_file_names,
}

train_data_file = "./train_data.json"
with open(train_data_file, "w") as f:
    json.dump(train_data, f)

# Print information
print("Training Dataset length:", len(train_dataset))
print("Validation Dataset length:", len(val_dataset))
print(f"Validation data saved to {val_data_file}")
print(f"Training data saved to {train_data_file}")

In [ ]:
import pandas as pd
import json

# File paths
val_data_file = "./val_data.json"
train_data_file = "./train_data.json"

# Load validation data from JSON file
with open(val_data_file, "r") as f:
    loaded_val_data = json.load(f)

loaded_val_indices = loaded_val_data["val_indices"]
loaded_val_file_names = loaded_val_data["file_names"]

# Create a DataFrame for validation data
val_data_df = pd.DataFrame({
    "val_index": loaded_val_indices,
    "filename": loaded_val_file_names
})

# Display the Validation DataFrame
print("Validation DataFrame:")
print(val_data_df.head())

# Save validation data to CSV
val_data_df.to_csv("./val_data.csv", index=False)
print("Validation data saved to ./val_data.csv")

# Load training data from JSON file
with open(train_data_file, "r") as f:
    loaded_train_data = json.load(f)

loaded_train_indices = loaded_train_data["train_indices"]
loaded_train_file_names = loaded_train_data["file_names"]

# Create a DataFrame for training data
train_data_df = pd.DataFrame({
    "train_index": loaded_train_indices,
    "filename": loaded_train_file_names
})

# Display the Training DataFrame
print("Training DataFrame:")
print(train_data_df.head())

# Save training data to CSV
train_data_df.to_csv("./train_data.csv", index=False)
print("Training data saved to ./train_data.csv")


In [ ]:
len(train_dataset)

In [ ]:
len(train_dataset[:10])

In [ ]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, train_dataset, val_dataset, crit, cost_norm, args)


In [ ]:
best_model_path

In [ ]:
len(loaded_val_indices[:10])

In [ ]:
len(val_embeddings)

In [ ]:
import pandas as pd
import numpy as np

# Assume these are loaded or computed
# val_embeddings: List or numpy array of embeddings (e.g., shape [num_examples, embedding_dim])
# loaded_val_indices: List of validation indices
# loaded_file_names: List of validation file names

# Convert val_embeddings to a numpy array if it's not already
val_embeddings = np.array(val_embeddings)

# Ensure the size of all inputs matches
if len(loaded_val_indices) != len(loaded_val_file_names) or len(loaded_val_indices) != len(val_embeddings):
    raise ValueError("The sizes of validation indices, filenames, and embeddings must match.")

# Create a DataFrame
val_data_df = pd.DataFrame({
    "val_index": loaded_val_indices,
    "filename": loaded_val_file_names,
    "embedding": list(val_embeddings)  # Store embeddings as a list of numpy arrays
})

# Save the DataFrame to a CSV file
csv_file_path = "validation_embeddings.csv"
val_data_df.to_csv(csv_file_path, index=False)

# Optionally save embeddings in a separate binary file for efficient storage
embeddings_file_path = "validation_embeddings.npy"
np.save(embeddings_file_path, val_embeddings)

print(f"Validation DataFrame created and saved to {csv_file_path}")
print(f"Embeddings saved to {embeddings_file_path}")

# Print the first few rows for verification
print(val_data_df.head())


In [ ]:
val_data_df.head()

In [ ]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


In [ ]:
print(f"First validation embedding shape: {val_embeddings[0]}")

In [ ]:
type(val_embeddings[0])

In [ ]:
print(f"First validation embedding shape: {val_embeddings[1]}")

In [ ]:
len(val_embeddings[0])

In [18]:
# import numpy as np

# # Disable truncation
# np.set_printoptions(threshold=np.inf, linewidth=1000)

# for i, emb in enumerate(val_embeddings):
#     print(f"Full Embedding {i}:\n{emb}")


In [ ]:
import json
import numpy as np

# Ensure embeddings are numpy arrays
train_embeddings = np.array(train_embeddings)
val_embeddings = np.array(val_embeddings)

# Paths for saving the JSON files
train_embeddings_file = "./train_embeddings.json"
val_embeddings_file = "./val_embeddings.json"

# Create dictionaries with file names as keys and embeddings as values
train_embedding_dict = {train_file_name: embedding.tolist() for train_file_name, embedding in zip(loaded_train_file_names, train_embeddings)}
val_embedding_dict = {val_file_name: embedding.tolist() for val_file_name, embedding in zip(loaded_val_file_names, val_embeddings)}

# Save training embeddings to JSON
with open(train_embeddings_file, "w") as f:
    json.dump(train_embedding_dict, f)

# Save validation embeddings to JSON
with open(val_embeddings_file, "w") as f:
    json.dump(val_embedding_dict, f)

print(f"Train embeddings saved to {train_embeddings_file}")
print(f"Validation embeddings saved to {val_embeddings_file}")

In [ ]:
import pandas as pd
import json

# Path to the JSON file
val_embeddings_file = "./val_embeddings.json"  # Replace with train_embeddings.json for training data

# Load the JSON file
with open(val_embeddings_file, "r") as f:
    embeddings_data = json.load(f)

# Convert JSON data to a DataFrame
val_embeddings_df = pd.DataFrame({
    "file_name": list(embeddings_data.keys()),
    "embedding": list(embeddings_data.values())
})

# Display the DataFrame
print(val_embeddings_df.head())

# Optionally save it to a CSV
val_embeddings_df.to_csv("val_embeddings.csv", index=False)
print("Validation embeddings saved to val_embeddings.csv")
